# WIBARAB FeatureDB Validation

* Install & set up dependencies
* extract schematron from RNG and transform to XSLT
* run schematron-xslt on files

In [44]:
import io
import os
import requests
import pathlib
import re
import sys
import json
import pandas as pd

from pathlib import Path
from urllib.parse import urlsplit
from saxonpy import PySaxonProcessor, PyXdmValue
from zipfile import ZipFile
from lxml import isoschematron, etree

In [2]:
tmpDir = "tmp"
libDir = "lib"
os.makedirs(tmpDir, exist_ok=True)


# the root of the git repository
dataHome = "../.."

# rng schema
rngSchema = dataHome + "/803_RNG_Schematron/featuredb.rng"
# the path to the annotated TEI transcription files
manannot = dataHome + "/010_manannot"
# the path to the feature documents 
features = manannot + "/features"



proc = PySaxonProcessor(license=False)
# SaxonC 1.2.1 Python has many known bugs but isn't maintained anymore
# Many of the documented API specs are not working
print(proc.version)
proc.set_cwd(os.path.dirname(os.path.abspath('')))
print(proc.cwd)

Saxon/C 1.2.1 running with Saxon-HE 9.9.1.5C from Saxonica
/home/dschopper/data/WIBARAB/featuredb/080_scripts_generic


In [3]:
def downloadAndStore(url, force=False):   
    #  filename of the file to be downloaded
    fn = os.path.basename(url)
    # fn w/o extension
    basename = os.path.splitext(fn)[0]
    # extension 
    ext = os.path.splitext(fn)[1]
    dlFilePath = tmpDir + "/" + fn
    if not os.path.exists(dlFilePath) and not force == True:
        payload = requests.get(url).content
        open(dlFilePath, 'wb').write(payload)
    return dlFilePath

In [4]:
def downloadAndUnzip(url):    
    #  filename of the file to be downloaded
    fn = os.path.basename(url)
    # fn w/o extension
    basename = os.path.splitext(fn)[0]
    # extension 
    ext = os.path.splitext(fn)[1]
    
    if ext != ".zip":
        return "not a zip archive"
    else:
        zipFilePath = downloadAndStore(url)
        # the path where the content should be extracted to
        targetPath = libDir + "/" + basename
        
        
        payload = requests.get(url).content
        open(zipFilePath, 'wb').write(payload)
        ZipFile(zipFilePath).extractall(path=targetPath)
    
    return targetPath

In [5]:
def setupSchXSLT():
    # install schematron
    schDLURL = "https://github.com/schxslt/schxslt/releases/download/v1.9.5/schxslt-1.9.5-xslt-only.zip"
    schHome = downloadAndUnzip(schDLURL)
    schCompiler = schHome + "/schxslt-1.9.5/2.0/compile-for-svrl.xsl"
    if os.path.exists(schCompiler):
        return schCompiler
    else: 
        print("error: something went wrong, cannot locate file '" + schCompiler + "'")

In [6]:
def transform(s, xsl, o, parameters=[]):
    proc.set_configuration_property("xi", "on")
    saxon = proc.new_xslt_processor()
    saxon.set_source(file_name=os.path.abspath(s))
    for i in parameters:
        saxon.set_parameter(name=i, value=proc.make_string_value(parameters[i]))
    saxon.compile_stylesheet(stylesheet_file=os.path.abspath(xsl))
    saxon.set_output_file(os.path.abspath(o))
    saxon.transform_to_string()
    if saxon.exception_occurred():
        #print(getmembers(saxon))
        saxon.get_error_message(0)
        for i in range(saxon.exception_count()-1):
            print(saxon.get_error_message(0))
        print(os.path.abspath(s)+" - "+os.path.abspath(xsl)+" -> "+os.path.abspath(o)+" failed")
    return o

In [7]:
# extract a schematron document embedded in an rng schema
def extractSchematron(rng):
    rng2sch = setupRNG2Sch()
    sch = tmpDir + "/" + os.path.basename(rng) + ".sch"
    if not os.path.exists(sch):
        transform(rng, rng2sch, sch)
    return sch

In [8]:
# compile a schematron document to an XSLT stylesheet
def compileSchematron(sch):
    outputPath = tmpDir + "/" + os.path.basename(sch) + ".xsl"
    schCompiler = setupSchXSLT()
    
    transform(sch, schCompiler, outputPath)
    if os.path.exists(outputPath):
        return outputPath
    else: 
        print("error: something went wrong, cannot locate file '" + outputPath + "'")

In [9]:
# prepare rng2sch styelsheet
# returns path to xsl
def setupRNG2Sch():
    RNG2SchtrDL = "https://raw.githubusercontent.com/Schematron/schematron/master/trunk/converters/code/ToSchematron/ExtractSchFromRNG.xsl"
    rng2sch = downloadAndStore(RNG2SchtrDL)
    # tweak XSLT 
    with open(rng2sch) as inputfile:
        lines = inputfile.read()
    lines = lines.replace( 'http://www.ascc.net/xml/schematron','http://purl.oclc.org/dsdl/schematron/')
    lines = lines.replace( '<sch:schema','<sch:schema queryBinding="xslt2"')
    
    with open(rng2sch, 'w') as file:
        file.writelines(lines)
    newPath = libDir+"/"+os.path.basename(rng2sch)
    os.replace(rng2sch, newPath)
    if os.path.exists(newPath):
        return newPath
    else:
        print("error: something went wrong, cannot locate file '" + newPath + "'")

In [55]:
# validate document doc against schematron schema sch
def schValidate(sch, path):
    out = tmpDir + "/validationReports/ " + os.path.basename(path)
    xsl = compileSchematron(sch)
    transform(path, xsl, out)
    report = etree.parse(out)
    success = report.findall("{http://purl.oclc.org/dsdl/svrl}successful-report")
    
    for s in success:
        location = s.attrib['location'].replace('Q{http://www.tei-c.org/ns/1.0}','')
        msg = s.find("{http://purl.oclc.org/dsdl/svrl}text").text
        return {
            "message": location + " " + msg,
            "source": path,
            "stage": "schematron",
            "exceptionType": s.tag
        }
    

In [11]:
def validate(path, rngSchema):
    validationErrors = []
    sch = extractSchematron(rngSchema)
    try:
        doc = etree.parse(path)
        # relaxng validation
        relaxng_doc = etree.parse(rngSchema)
        relaxng = etree.RelaxNG(relaxng_doc)
        relaxng.assertValid(doc)
        
        # schematron validation
        schErrors = schValidate(sch, path)
        validationErrors.append(schErrors)
    
    except etree.DocumentInvalid as e:
        valErrObj = {"message": str(e), "source": path, "stage" : "relaxng", "exceptionType": type(e).__name__}
        validationErrors.append(valErrObj)
        schValidate(sch, path)
        
        
    except etree.XMLSyntaxError as e:
        valErrObj = {"message": str(e), "source": path, "stage" : "parsing", "exceptionType": type(e).__name__}
        validationErrors.append(valErrObj)
    return validationErrors

In [56]:
validationErrors = []

for i in os.scandir(features):
    if i.name.endswith('.xml') and i.is_file():
        filename=os.path.basename(i)
        filepath=features + "/" + filename
        print("validating " + filepath)
        errs = validate(filepath, rngSchema)
        validationErrors = errs + validationErrors

validating ../../010_manannot/features/features_pron_sg_p2_c.xml
validating ../../010_manannot/features/feature_bound_pronoun_2ps_sg_f_after_vowel.xml
validating ../../010_manannot/features/feature_bound_pronoun_2ps_pl_m.xml
validating ../../010_manannot/features/feature_bound_pronoun_2ps_sg_c_after_vowel.xml
validating ../../010_manannot/features/features_pron_sg_p1_c.xml
validating ../../010_manannot/features/feature_bound_pronoun_3ps_sg_m_after_vowel.xml
validating ../../010_manannot/features/features_pron_pl_p3_f.xml
validating ../../010_manannot/features/feature_bound_pronoun_3ps_sg_m_after_consonant.xml
validating ../../010_manannot/features/feature_bound_pronoun_2ps_sg_c_after_consonant.xml
validating ../../010_manannot/features/features_q.xml
validating ../../010_manannot/features/features_pron_pl_p1_c.xml
validating ../../010_manannot/features/features_pron_pl_p3_c.xml
validating ../../010_manannot/features/features_pron_pl_p2_m.xml
validating ../../010_manannot/features/featu

In [57]:
df = pd.DataFrame(data=validationErrors)
df = df.fillna(' ').T
df = df.transpose()

errorReport = "tmp/validationReport.html"
with open(errorReport, 'w') as f:
    f.write(df.to_html())
